In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score

from keras.layers import Dense,Dropout,SimpleRNN,LSTM,Input
from keras.models import Sequential
from tensorflow.keras.models import Model

**Data Loading and Preprocessing**

In [ ]:
fpath=r'D:\Neeraj\SEM 7\archive\DOM_hourly.csv'

df=pd.read_csv(fpath)
df.head()

In [ ]:
df['Datetime']=pd.to_datetime(df['Datetime'])
df.set_index('Datetime',inplace=True)
df.head()

In [ ]:
df.plot()
plt.title('DOM hourly power consumption data - BEFORE NORMALIZATION')
plt.show()

In [ ]:
def normalise(df):
  scaler=MinMaxScaler()
  df['DOM_MW']=scaler.fit_transform(df['DOM_MW'].values.reshape(-1,1))
  return df
df=normalise(df)
df.shape

In [ ]:
df.plot()
plt.title('DOM hourly power consumption data - AFTER NORMALIZATION')
plt.show()

In [ ]:
def load_data(stock,seq_len):
  X_train = []
  y_train = []
  for i in range(seq_len, len(stock)):
        X_train.append(stock.iloc[i-seq_len : i, 0])
        y_train.append(stock.iloc[i, 0])
  X_test = X_train[110000:]
  y_test = y_train[110000:]

  X_train = X_train[:110000]
  y_train = y_train[:110000]

  X_train = np.array(X_train)
  y_train = np.array(y_train)
  X_test = np.array(X_test)
  y_test = np.array(y_test)

  X_train = np.reshape(X_train, (110000, seq_len, 1))
  X_test = np.reshape(X_test, (X_test.shape[0], seq_len, 1))

  return [X_train, y_train, X_test, y_test]

In [ ]:
seq_len = 20

X_train, y_train, X_test, y_test = load_data(df, seq_len)

print('X_train.shape = ',X_train.shape)
print('y_train.shape = ', y_train.shape)
print('X_test.shape = ', X_test.shape)
print('y_test.shape = ',y_test.shape)

**Building RNN Model**

In [ ]:
def create_rnn_model():

  input_layer=Input(shape=(X_train.shape[1],1))

  l1=SimpleRNN(40,activation='tanh',return_sequences=True)(input_layer)
  l2=Dropout(0.15)(l1)

  l3=SimpleRNN(40,activation='tanh',return_sequences=True)(l2)
  l4=Dropout(0.15)(l3)

  l5=SimpleRNN(40,activation='tanh',return_sequences=False)(l4)
  l6=Dropout(0.15)(l5)

  l7=Dense(1)(l6)

  model=Model(inputs=input_layer,outputs=l7)
  model.summary()
  return model

rnn_model=create_rnn_model()
rnn_model.compile(optimizer="adam",loss="MSE")
rnn_model.fit(X_train, y_train, epochs=10,batch_size=1000)


In [ ]:
rnn_predictions = rnn_model.predict(X_test)
rnn_score = r2_score(y_test,rnn_predictions)
print("R2 Score of RNN model = ",rnn_score)

In [ ]:
def plot_predictions(test, predicted, title):
    plt.plot(test, color='blue',label='Actual power consumption data')
    plt.plot(predicted, color='orange',label='Predicted power consumption data')
    plt.title(title)
    plt.xlabel('Time')
    plt.ylabel('Normalized power consumption scale')
    plt.legend()
    plt.show()

plot_predictions(y_test, rnn_predictions, "Predictions made by simple RNN model")

**Building LSTM Model**

In [ ]:
def create_lstm_model():

  input_layer=Input(shape=(X_train.shape[1],1))

  l1=LSTM(40,activation='tanh',return_sequences=True)(input_layer)
  l2=Dropout(0.15)(l1)

  l3=LSTM(40,activation='tanh',return_sequences=True)(l2)
  l4=Dropout(0.15)(l3)

  l5=LSTM(40,activation='tanh',return_sequences=False)(l4)
  l6=Dropout(0.15)(l5)

  l7=Dense(1)(l6)

  model=Model(inputs=input_layer,outputs=l7)
  model.summary()
  return model

lstm_model=create_lstm_model()
lstm_model.compile(optimizer="adam",loss="MSE")
lstm_model.fit(X_train, y_train, epochs=10,batch_size=1000)


In [ ]:
lstm_predictions = lstm_model.predict(X_test)
lstm_score = r2_score(y_test, lstm_predictions)
print("R^2 Score of LSTM model = ",lstm_score)

plot_predictions(y_test, lstm_predictions, "Predictions made by LSTM model")

**Comparing RNN and LSTM Models**

In [ ]:
plt.plot(y_test, c="orange", label="Original values")
plt.plot(lstm_predictions, c="red", label="LSTM predictions")
plt.plot(rnn_predictions, c="green", label="RNN predictions")
plt.legend()
plt.title("Predictions vs actual data")
plt.show()